The code requires FEniCSx---see the README for details.

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')

In [ ]:
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import numpy as np
from dolfinx.fem import Constant, Function, FunctionSpace
from dolfinx.mesh import create_interval
from misc import get_stress, interp
from mpi4py import MPI
from params import eta, H0, nt, nz, phi0, w0, zeta
from petsc4py import PETSc
from solvers import full_solve, vel_solve
from ufl import FiniteElement, MixedElement

Define domain:

In [ ]:
# generate mesh, initial domain is has a scaled height of 1 
domain = create_interval(MPI.COMM_WORLD,nz,[0,1])

In [ ]:
P1 = FiniteElement('P',domain.ufl_cell(),1)     
element = P1*P1
V = FunctionSpace(domain,element)   
initial = Function(V)
initial.sub(1).interpolate(lambda x:phi0+0*x[0])

In [ ]:
# # Dirichlet condition: set compaction rate (velocity) at the top
bc_top = {'type': 'velocity', 'value': -1}

# # Neumann condition: set the load (stress) at the top
# bc_top = {'type': 'stress', 'value': 1e1*H0/(w0*((4./3.)*eta + zeta))}

In [ ]:
w_i = vel_solve(domain,phi0,bc_top)
initial.sub(0).interpolate(w_i)

In [ ]:
w,phi,sigma,z = full_solve(domain,initial,bc_top)

Plot the solution:

In [ ]:
# time array for plotting
t = np.outer(np.linspace(0,1,nt),np.ones(nz+1))
sigma0 = (1-phi0)*w[0,-1]
d = int(sigma.min()/sigma0)

plt.figure(figsize=(8,4))
plt.subplot(131)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=1)
plt.contourf(t,z,-w,cmap='Blues',levels=np.linspace(0,1,100),extend='both')
plt.ylabel(r'$z$',fontsize=18)
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,1,5))
cbar.set_label(r'$w\,/\,w_0$',fontsize=24)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_xticklabels([0,0.25,0.5,0.75,1])

plt.subplot(132)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=1)
plt.contourf(t,z,sigma/sigma0,cmap='Blues',levels=np.linspace(0,d,100),extend='both')
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.gca().yaxis.set_ticks([])
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,d,5))
cbar.set_label(r'$\Sigma$',fontsize=24)
cbar.ax.tick_params(labelsize=12)
#cbar.ax.set_xticklabels([0,0.25,0.5,0.75,1])

plt.subplot(133)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=1)
plt.contourf(t,z,phi/phi0,cmap='Blues',levels=np.linspace(0,1,100),extend='both')
plt.ylabel(r'$z$',fontsize=18)
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.gca().yaxis.set_label_position("right")
plt.gca().yaxis.tick_right()
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,1,5))
cbar.set_label(r'$\phi\,/\,\phi_0$',fontsize=24)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_xticklabels([0,0.25,0.5,0.75,1])
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
dH = -(z[:,-1] - z[0,-1])
sigma_H = -sigma[:,-1]
sigma_sc = (w0*((4./3.)*eta + zeta))/H0/1e3
dH_sc = H0*1e3

plt.figure(figsize=(8,6))
plt.plot(dH*dH_sc,sigma_H*sigma_sc,color='royalblue',linewidth=3)
plt.ylabel(r'$\Sigma$ (kPa)',fontsize=20)
plt.xlabel(r'$\delta$ (mm)',fontsize=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()
plt.close()